# SavingsPlan - Analysis of Customer Retention [Data Acquisition]

## Setup

### Libraries

In [ ]:
import psycopg2
import numpy as np
import pandas as pd
import datetime as dt
# import cx_Oracle as ora
import missingno as msno
import scipy.stats as stats
import seaborn as sns; sns.set()
sns.palplot(sns.light_palette("green"))
import matplotlib.pyplot as plt
%matplotlib inline
from warnings import filterwarnings
filterwarnings('ignore')

### DB connection

In [ ]:
db_params = {'name': 'zoolake', 'user': 'dsadmin', 'pass': 'yH*3Y4@kqmM', 'port': 5432,
             'host':'dna-lake-prod-c-zoolake-cluster.cluster-c90xuwjhoktv.eu-central-1.rds.amazonaws.com'}

In [ ]:
conn_string = "dbname={dbname} user={user} password={password} host={host} port={port}" \
              .format(dbname=db_params['name'], user=db_params['user'], password=db_params['pass'], host=db_params['host'], port=db_params['port'])
conn = psycopg2.connect(conn_string)

## Data Acquisition

The analysis will be deployed per **site** and **customer**.

In [ ]:
site = 1  # Zooplus Germany (aprox 5000000 orders in 2 years)

The eligible customers will be those ones whose **first order** took place in a fixed range of **two years** (mid 2014 to mid 2016).

In [ ]:
s_date = "to_date('01/07/14', 'DD/MM/YY')" # Start date
e_date = "to_date('30/06/16', 'DD/MM/YY')" # End date
n_ord = 1   # First orders

In [ ]:
subquery = """ 
           SELECT DISTINCT o.o_k_id 
           FROM zooman.orders o 
           JOIN zooman.order_sales_profile osp 
           ON osp.osp_o_id = o.o_id 
           WHERE o.o_s_id = {site_id} 
           AND (o.o_datum >= {start_date} and o.o_datum <= {end_date}) 
           AND osp.OSP_KO_COUNT_NET = {n_order} 
           """
subquery = subquery.format(site_id=site, start_date=s_date, end_date=e_date, n_order=n_ord)

Now, for previously selected customers, we get all the **valid orders** during a total period of **four years** (mid 2014 to mid 2018).

In [ ]:
o_st_1 = 'L'; o_st_2 = 'VK'  # Valid orders
intrvl = '2'   # 2 minutes interval

In [ ]:
query = """
        SELECT o.O_ID, o.O_K_ID, o.O_DATUM, o.O_GWERT O_SALES_VALUE, o.O_S_ID,
        osp.OSP_KO_COUNT_NET, osp.OSP_3_LEAD_P_PG_ID, osp.OSP_3_LEAD_P_PG_SHARE, 
        osp.OSP_4_LEAD_P_PG_ID, osp.OSP_4_LEAD_P_PG_SHARE,
        czp.CZP_START_DATE, czp.CZP_EXPIRY_DATE, czp.CZP_ZTQ_ID, czp.CZP_PERCENTAGE, czp.CZP_PRICE, czp.CZP_DURATION
        FROM zooman.orders o
        JOIN zooman.order_sales_profile osp
        ON osp.OSP_O_ID = o.O_ID
        LEFT OUTER JOIN zooman.customers_zooprime_plans czp
        ON o.o_k_id = czp.CZP_K_ID
        AND czp.CZP_EXPIRY_DATE > O_DATUM
        AND czp.CZP_START_DATE <= (O_DATUM + interval '{interval}' minute)
        WHERE o.o_s_id = {site_id}
        AND (o.o_status = '{o_status_1}' OR o.o_status = '{o_status_2}')
        AND o.o_k_id IN ({subquery}) 
        """
query = query.format(site_id=site, o_status_1=o_st_1, o_status_2=o_st_2, interval=intrvl, subquery=subquery)

In [ ]:
# Load the data taking into account that certain columns are dates
col_dates = ['o_datum', 'czp_start_date', 'czp_expiry_date']
df = pd.read_sql(query, con = conn, parse_dates=col_dates)

In [ ]:
# Convert to integers those columns previously loaded as real
col_names_with_ids = ['o_id', 'o_k_id', 'o_s_id']
df[col_names_with_ids] = df[col_names_with_ids].astype(int)

In [ ]:
df.to_pickle('./data/df_site_1_initial_orders_2014-2018.pkl')

In [ ]:
# input_df = pd.read_pickle("./data/df_site_1_initial_orders_2014-2018.pkl")